## Import data

In [79]:
import pandas

In [81]:
accidents = pandas.read_csv("../datasets/project/road safety/road-safety-data-accidents-2016.csv")
casualties = pandas.read_csv("../datasets/project/road safety/road-safety-data-casualties-2016.csv")
vehicles = pandas.read_csv("../datasets/project/road safety/road-safety-data-vehicles-2016.csv")

## Plot data

In [83]:
import folium

In [84]:
# https://stackoverflow.com/questions/42756934/how-to-plot-lat-and-long-from-pandas-dataframe-on-folium-map-group-by-some-label
# Map centered at Dundee (or, where Google tells me Dundee is...)
colors = {'Slight': 'green', 'Serious': 'orange', 'Fatal': 'red'}
map_osm = folium.Map(location=[56.4620, -2.9707], zoom_start=12)
accidents.apply(lambda row:folium.CircleMarker(location=[row["Latitude"], row["Longitude"]], 
                                               radius=10, 
                                               fill_color=colors[row["Accident_Severity"]],
                                               fill = True,
                                               stroke=False,
                                               fill_opacity=0.5,
                                              popup = row["Accident_Index"])
                                             .add_to(map_osm), axis=1)
map_osm

## Thoughts
- All accidents seem to be on roads, except 2016930000001 (rural)
    - Could be tested by comparing overlap with OSM road data
    - Unsurprising
- Only fatality is at a crossing
    - Unsurprising
- Most accidents seem to be in high-traffic areas (from walking around city)
    - Any data to confirm?

In [85]:
fatal_acc = accidents.loc[accidents["Accident_Index"] == "2016930000604"]
fields = fatal_acc.columns.values
row = fatal_acc.as_matrix()

In [86]:
for x in range(0, len(fields)):
    print("{}: {}".format(fields[x], row[0][x]))

Accident_Index: 2016930000604
Location_Easting_OSGR: 338869
Location_Northing_OSGR: 731840
Longitude: -2.993967
Latitude: 56.474751
Police_Force: Tayside
Accident_Severity: Fatal
Number_of_Vehicles: 1
Number_of_Casualties: 1
Date: 17/11/2016
Day_of_Week: Thursday
Time: 11:05
Local_Authority_(District): Dundee City
Local_Authority_(Highway): Dundee City
1st_Road_Class: C
1st_Road_Number: 201
Road_Type: Single carriageway
Speed_limit: 30
Junction_Detail: Mini-roundabout
Junction_Control: Give way or uncontrolled
2nd_Road_Class: C
2nd_Road_Number: 222
Pedestrian_Crossing-Human_Control: None within 50 metres 
Pedestrian_Crossing-Physical_Facilities: Central refuge
Light_Conditions: Daylight
Weather_Conditions: Raining no high winds
Road_Surface_Conditions: Wet or damp
Special_Conditions_at_Site: None
Carriageway_Hazards: None
Urban_or_Rural_Area: Urban
Did_Police_Officer_Attend_Scene_of_Accident: Yes
LSOA_of_Accident_Location: nan


# Correlations
1. How does weather type affect severity of accident?
    - Would expect serious events to be during bad weather
2. How does time of day affect severtity of accident?
    - Would expect serious events to be during the night
    